In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood
from inference_toolbox.sampler import Sampler
from inference_toolbox.visualiser import Visualiser
from inference_toolbox.domain import Domain

from data_processing.get_data import get_data
# from gaussian_processor.gaussian_processor import GaussianProcessor

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
# gp = GaussianProcessor(averaged_df, kernel = 'matern_white', data_norm='sqrt')
# training_data, test_data = gp.train_test_split()
# print(training_data)
# gp.train_gp(training_data)
# gp.test(test_data)
# grid = box_gridder.get_grid([10,10,10])
# gp.predict_from_gp(grid, threeD=True, save=True, log_results=True)
# gp.animate() 

import os

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

In [3]:
# data_type = 'dummy'
# data_params = {
#     'data_type': 'dummy',
#     'data_path': 'data',
#     'sigma': 1,
#     'model_select': 'log_gpm_alt_norm',
#     'noise_dist': 'gaussian',
#     'model': {
#         'model_params':{
#             'H': 10
#         },
#         'inference_params':{
#             'I_y': 0.1,
#             'I_z': 0.1,
#             'Q': 3e13
#         },
#     },
#     'domain': {
#         'domain_select': 'cone_from_source_z_limited', 
#         'resolution': 20,
#         'domain_params':{
#             'r': 100,
#             'theta': np.pi/8,
#             'source': [0,0,10]}
#     },
#     'output_header': 'Concentration'
# }

data_type = 'real_gridded'
data_params = {
    'data_type': 'real_gridded',
    'output_header': 'Concentration'
}

data = get_data(data_type, data_params)

/project/data_processing/box_gridder.py:89: RuntimeWarning: invalid value encountered in divide
  averages = sums/counts


In [4]:
# training_data, testing_data = train_test_split(averaged_df, test_size=0.2)
training_data, testing_data = train_test_split(data, test_size=0.2)

In [5]:
# Initialize parameter series
params = pd.Series({},dtype='float64')

# Parameter Assignment
I_y = Parameter(name = 'I_y', init_val=0.1, prior_select="gamma")
I_y.add_prior_param("mu",0.1)
I_y.add_prior_param("sigma",0.1)
params['I_y'] = I_y

I_z = Parameter(name = 'I_z', init_val=0.1, prior_select="gamma")
I_z.add_prior_param("mu",0.1)
I_z.add_prior_param("sigma",0.1)
params['I_z'] = I_z

Q = Parameter(name = 'Q', init_val=3e13, prior_select="gamma")
Q.add_prior_param("mu",3e13)
Q.add_prior_param("sigma",1e13)
params['Q'] = Q

sigma = Parameter(name = 'sigma', init_val=1, prior_select="gamma")
sigma.add_prior_param("mu",1)
sigma.add_prior_param("sigma",1)
params['sigma'] = sigma

# Model Assignment
model = Model('log_gpm_alt_norm')
model.add_model_param("H",10)

# Likelihood function assigmnent
likelihood = Likelihood("gaussian")
# likelihood.add_likelihood_param("sigma",0.5)
# likelihood.add_likelihood_param("lambda_1",1)
# likelihood.add_likelihood_param("lambda_2",0.05)

num_samples = 1000

data_path = 'results/gridded_drone_data_1/inference/general_instances'

# Initialize the sampler
# sampler = Sampler(params, model, likelihood, training_data, num_samples, show_sample_info = True, n_chains=4, thinning_rate=1,data_path = 'results/simulated_data_1/inference/general_instances')
sampler = Sampler(params, 
                  model, 
                  likelihood, 
                  training_data, 
                  num_samples, 
                  show_sample_info = True, 
                  n_chains=4, 
                  thinning_rate=1,
                  data_path = data_path)

# Sample the parameters
params_samples, chain_samples, fields = sampler.sample_all()

In [6]:
# visualiser = Visualiser(testing_data, 
#                         params_samples, 
#                         model, 
#                         sampler.hyperparams, 
#                         chain_samples=chain_samples,
#                         fields = fields, 
#                         previous_instance = sampler.instance, 
#                         data_path = 'results/simulated_data_1/inference/general_instances', 
#                         actual_values=[
#                             data_params['model']['inference_params']['I_y'], 
#                             data_params['model']['inference_params']['I_z'],
#                             data_params['model']['inference_params']['Q'],
#                             data_params['sigma']])

visualiser = Visualiser(testing_data, 
                        params_samples, 
                        model, 
                        sampler.hyperparams, 
                        chain_samples=chain_samples,
                        fields = fields, 
                        previous_instance = sampler.instance, 
                        data_path = data_path)

visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 100)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

Exception: Chain samples file does not exist!